# HIV Analysis

Here we combine HIV sequence data from the [Los Alamos National Laboratory HIV Sequence Database](www.hiv.lanl.gov) and immunological data to investigate HIV evolution across 14 individuals. This data is contained in the `data/HIV/` directory. Supplementary scripts for data processing are stored in `HIV.py`.

We downloaded sequence data for all patients and aligned them with the [HXB2 reference sequence](https://www.hiv.lanl.gov/components/sequence/HIV/asearch/query_one.comp?se_id=K03455) and the corresponding [clade consensus sequence](https://www.hiv.lanl.gov/content/sequence/NEWALIGN/align.html#consensus) using the LANL tool [HIValign](https://www.hiv.lanl.gov/content/sequence/VIRALIGN/viralign.html). We recorded immunological data for these patients in `.csv` files for analysis.

Additonal note (August.23.2023) How to get HIValign.fasta files. 
The steps to generate the files are just 
1. download all relevant sequences from LANL (xxx_gap_squeeze.fasta). 
2. Add the consensus sequence to the file (xxx_gap_squeeze_cons.fasta). 
3. Align the sequences together with the consensus sequence (xxx-HIValign.fasta). 
4. It looks like there is some subsequent processing here to remove gaps, but whatever this is, it should be in the notebook — this was not done by any other script.




### Sequence processing

In the scripts below, we first process the HIV sequence data by
1. associating times with sequences and removing all sequences for which no collection time is available,
2. clipping full-length sequences down to the heavily-sequenced half-genome regions,
3. removing sequences with excess gaps in the targeted region, which primarily consist of sequences that cover the opposite half-genome region,
4. removing sites that are >= 95% gaps,
5. time-ordering sequences,
6. removing time points that are > 300 days from the previous time point and/or have < 4 sequences available at that time point, in order to ensure minimum quality of the correlations,
6. imputing ambiguous nucleotides with the most frequently observed nucleotide at the same site (and optionally, imputing gaps at the sequence edges due to different sequencing windows; with or without imputation, gaps at the edges will be flagged), and
7. saving the data in a format readable by the MPL program.


### Load libraries and define global variables

In [1]:
print('This notebook was prepared using:')

import os
import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import HIV

# GLOBAL VARIABLES

NUC = ['-', 'A', 'C', 'G', 'T']
PRO = ['-', 'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
       'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
REF = NUC[0]
CONS_TAG = 'CONSENSUS'
HXB2_TAG = 'B.FR.1983.HXB2-LAI-IIIB-BRU.K03455.19535'
C_PPTS = ['703010505', '703010848']
#C_PPTS = ['706010164']
# # Code Ocean directories
# HIV_DIR = '../data/HIV'
# MPL_DIR = 'MPL'
# HIV_MPL_DIR = '../data/HIV/MPL'

# GitHub directories
HIV_DIR = 'data/HIV'
MPL_DIR = 'src/MPL'
HIV_MPL_DIR = 'src/MPL/HIV'

# Sequence processing
MAX_GAP_NUM  = 200
MAX_GAP_FREQ = 0.95
MIN_SEQS     = 4
MAX_DT       = 300

This notebook was prepared using:
python version 3.12.0 | packaged by Anaconda, Inc. | (main, Oct  2 2023, 17:29:18) [GCC 11.2.0]
numpy version 1.26.4
pandas version 2.2.1


### Process sequences

In [3]:
# Load the sequence coverage for each patient
df_range = pd.read_csv('%s/range.csv' % (HIV_DIR), comment='#', memory_map=True)

# Load the list of transmitted/founder (TF) sequences
df_TF = pd.read_csv('%s/TF.csv' % (HIV_DIR), comment='#', memory_map=True)

# Load the list of exposed residues on Env
df_exposed = pd.read_csv('%s/exposed.csv' % (HIV_DIR), comment='#', memory_map=True)

# Load the set of epitopes targeted by patients
df_epitope = pd.read_csv('%s/epitopes.csv' % (HIV_DIR), comment='#', dtype={'ppt': str}, memory_map=True)


# Association times with sequences, remove all sequences (except HXB2 and consensus)
# that do not have identifiable times; TF sequence is treated as day 0 if available

for ppt in set(df_range.ppt):
    HIV.filter_sequences(ppt, '%s/raw/%s-HIValign.fasta' % (HIV_DIR, ppt), check_time=True)
    
    
# Store baseline number of nonsynonymous mutations in CD8+ T cell epitopes, 
# nonsynonymous reversions (total, in epitopes, and outside epitopes), and synonymous mutations
BASELINE_NS_EPITOPE = 0
BASELINE_NS_REV = 0
BASELINE_NS_REV_NONEPI = 0
BASELINE_SYN = 0
BASELINE_TOT = 0

# Select and process alignment subsets for analysis
g = open('%s/run.sh' % (MPL_DIR), 'w')
g.write('g++ src/main.cpp src/inf.cpp src/io.cpp -O3 -march=native -lgslcblas -lgsl -o bin/mpl\n')

for df_range_iter, df_range_entry in df_range.iterrows():
    
#     # Filter by subtype
#     if df_range_entry.ppt not in C_PPTS:
#         continue
    
    # Read in MSA
    ppt = df_range_entry.ppt
    msa, tag = HIV.get_MSA('%s/interim/%s-HIValign-filtered.fasta' % (HIV_DIR, ppt), noArrow=True)
    print('%s' % df_range_entry.tag)
    
    # Clip MSA to specified range
    msa = HIV.clip_MSA(df_range_entry.start, df_range_entry.end, msa, tag)
    
    # Filter sequences for quality (maximum #excess gaps)
    current_msa, current_tag = HIV.filter_excess_gaps(msa, tag, MAX_GAP_NUM, MAX_GAP_FREQ, verbose=True)
    
    # Put sequences in time order (first entries are HXB2, consensus)
    current_msa, current_tag = HIV.order_sequences(current_msa, current_tag)
    
    # Locate or compute TF sequence
    TF_accession = df_TF[df_TF.ppt==ppt].iloc[0].TF
    TF_sequence = HIV.get_TF(current_msa, current_tag, TF_accession)
    
    temp_msa = [current_msa[list(current_tag).index(HXB2_TAG)], TF_sequence]
    temp_tag = [HXB2_TAG, '%s-TF' % ppt]
    HIV.save_MSA(temp_msa, temp_tag, '%s/interim/TF-%s' % (HIV_DIR, df_range_entry.tag))
    
    # Impute ambiguous nucleotides and gaps at start/end of alignment
    current_msa = HIV.impute_ambiguous(current_msa, current_tag, start_index=2, verbose=True, impute_edge_gaps=False)
    
    # Map between local (alignment) index, index of polymorphic sites, and HXB2
    # Track location of known T cell epitopes, TF sequence, and exposed residues of Env
    poly_sites, current_msa, current_tag = HIV.create_index(current_msa, current_tag, TF_sequence, 
                                           current_msa[list(current_tag).index(CONS_TAG)], 
                                           current_msa[list(current_tag).index(HXB2_TAG)],
                                           df_range_entry.start, MIN_SEQS, MAX_DT, 
                                           df_epitope[df_epitope.ppt==ppt], df_exposed, 
                                           '%s/processed/%s-index.csv' % (HIV_DIR, df_range_entry.tag), 
                                           return_polymorphic=True, return_truncated=True)
    
    # Save alignment to format readable by MPL
    poly_states, poly_times = HIV.save_MPL_alignment(current_msa, current_tag, 
                                                     '%s/%s-poly-seq2state.dat' % (HIV_MPL_DIR, df_range_entry.tag), 
                                                     polymorphic_sites=poly_sites, return_states=True)
    
    # Save allele frequency trajectories for polymorphic sites
    # Later this file will be edited to include inferred selection coefficients
    df_index = pd.read_csv('%s/processed/%s-index.csv' % (HIV_DIR, df_range_entry.tag), comment='#', memory_map=True)
    HIV.save_trajectories(poly_sites, poly_states, poly_times, TF_sequence, df_index,
                          '%s/interim/%s-poly.csv' % (HIV_DIR, df_range_entry.tag))
    HIV.save_trajectories_alternate(poly_sites, poly_states, poly_times, TF_sequence, df_index,
                          '%s/interim/%s-poly-alternate.csv' % (HIV_DIR, df_range_entry.tag))
    
    # Get baseline number of possible mutations in different categories
    if ppt not in ['cap256']:
        df_poly = pd.read_csv('%s/interim/%s-poly.csv' % (HIV_DIR, df_range_entry.tag), comment='#', memory_map=True)
        temp_ns_epitope, temp_ns_rev, temp_ns_rev_ne, temp_syn = HIV.get_baseline(TF_sequence, df_index, df_poly)
        BASELINE_NS_EPITOPE += temp_ns_epitope
        BASELINE_NS_REV += temp_ns_rev
        BASELINE_NS_REV_NONEPI += temp_ns_rev_ne
        BASELINE_SYN += temp_syn
        BASELINE_TOT += 3 * len(df_index) + np.sum(df_poly.nucleotide=='-')
    
    # Save MSA
    HIV.save_MSA(current_msa, current_tag, '%s/interim/%s' % (HIV_DIR, df_range_entry.tag))
    
    # Record instructions for running MPL on this data
    io_str = '%s-poly-seq2state' % df_range_entry.tag
    run_str = ('./bin/mpl -d HIV -i %s.dat -o %s-MPL.dat -g 5e4 -N 1e4 -m Zanini-extended.dat -sc %s -sn %s\n'+
               './bin/mpl -d HIV -i %s.dat -o %s-SL.dat  -g 5e4 -N 1e4 -m Zanini-extended.dat -nc\n')
    g.write(run_str % (io_str, io_str, 'covariance-'+io_str+'.dat', 'numerator-'+io_str+'.dat', io_str, io_str))

g.close()


# Print baseline numbers of possible mutations in different categories
print('\nOut of %d possible mutations,' % BASELINE_TOT)
print('\t%d (%.2f%%) nonsynonymous in epitope' 
      % (BASELINE_NS_EPITOPE, 100*BASELINE_NS_EPITOPE/BASELINE_TOT))
print('\t%d (%.2f%%) nonsynonymous reversion'
      % (BASELINE_NS_REV, 100*BASELINE_NS_REV/BASELINE_TOT))
print('\t%d (%.2f%%) nonsynonymous reversion, in epitope' 
      % (BASELINE_NS_REV-BASELINE_NS_REV_NONEPI, 100*(BASELINE_NS_REV-BASELINE_NS_REV_NONEPI)/BASELINE_TOT))
print('\t%d (%.2f%%) nonsynonymous reversion, nonepitope' 
      % (BASELINE_NS_REV_NONEPI, 100*BASELINE_NS_REV_NONEPI/BASELINE_TOT))
print('\t%d (%.2f%%) synonymous'
      % (BASELINE_SYN, 100*BASELINE_SYN/BASELINE_TOT))

data/HIV/raw/703010848-HIValign.fasta
data/HIV/raw/703010848-accession2time.dat
data/HIV/raw/703010505-HIValign.fasta
data/HIV/raw/703010505-accession2time.dat
703010505-3
	selected 602 of 623 sequences with <200 gaps in excess of consensus
	removed 239 of 2891 sites with >95% gaps
	exchanged C for Y in sequence 70, site 122
	exchanged C for M in sequence 70, site 157
	exchanged C for M in sequence 58, site 875
	exchanged C for M in sequence 77, site 875
	exchanged C for Y in sequence 170, site 935
	exchanged A for R in sequence 100, site 954
	exchanged C for Y in sequence 220, site 1029
	exchanged A for R in sequence 112, site 1272
	exchanged A for R in sequence 117, site 1272
	exchanged C for Y in sequence 58, site 1273
	exchanged C for Y in sequence 112, site 1273
	exchanged C for Y in sequence 117, site 1273
	exchanged C for M in sequence 220, site 1285
	exchanged A for M in sequence 220, site 1287
	exchanged A for R in sequence 170, site 1291
	exchanged C for Y in sequence 102, si